# Base Imports

In [1]:
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
from cv2 import resize
import skimage as sk
from skimage import transform
import skimage.io as skio
import numpy as np


# Utils

In [2]:
def get_image(path):
    im = plt.imread(path)
    im = im.astype(float)
    return im

def show(im, figsize = 10, cmap=None):
    plt.figure(figsize=(figsize,figsize))
    plt.imshow(im, cmap=cmap)
    
def resize_preserve(img, shape):
    """
    Here, this is to crop without warping the input image
    """
    cv2_shape = np.array([img.shape[1], img.shape[0]])
    initial_upscale = max(shape[0]/cv2_shape[0], shape[1]/cv2_shape[1])
    img = resize(img, (cv2_shape * initial_upscale).astype(int))
    cv2_shape = np.array([img.shape[1], img.shape[0]])

    # Crop in x direction if needed
    xdiff = img.shape[1] - shape[0]
    if xdiff:
        img = img[:,xdiff//2:img.shape[1]-xdiff//2]
    
    # Crop in y direction if needed
    ydiff = img.shape[0] - shape[1]
    if ydiff:
        img = img[ydiff//2:img.shape[0]-ydiff//2]
    
    return img


# Correspondences

## Imports

In [3]:
import matplotlib.cm as cm
from scipy.spatial import Delaunay

## Code

In [4]:
def get_correspondences(images, dims = (1, 1), timeout = -1, include_edges = False, cmap = None, mesh = False, rainbow = True):
    """
    Takes in a list of images, returns a list of (x, y) keypoints for each image, in order
    
    Returns a list of keypoints in the shape (images, num_keypoints, 2 (since x and y) )
    
    Optional Args:
    include_edges - boolean for whether or not to count the edges of the image as keypoints automatically
    dims - the number of images, triangulations, and points to display as we decide to keep going or quit
    timeout - the time after which no click closes out the program
    cmap - color map option is nice if you're dealing with grayscale images
    mesh - Whether or not to display a mesh from the keypoints
    rainbow - whether or not to color the points with a rainbow
    """
    
    # Make sure all of our images are properly sized
    for i, image in enumerate(images):
        if len(image.shape) < 3:
            images[i] = np.stack((image, image, image), axis = -1)
    
    # Initialize our list of keypoints as empty of the edges of the images
    images_keypoints = [[] for image in images]
    if include_edges:
        images_keypoints = [[(0,0), (0, image.shape[0]-1), (image.shape[1]-1, 0), 
                             (image.shape[1]-1, image.shape[0]-1)]
                            for image in images]
    
    while True:
        # Display all the images that we have and the keypoints, and traingulations
        plt.clf()
        show_correspondences(images, images_keypoints, dims, cmap, mesh, rainbow, 
                             title = "Click to add another round of points, or press enter")
            
        # See if we want to add more keypoints
        if plt.waitforbuttonpress():
            plt.close()
            return images_keypoints
        plt.clf()
        
        # Start adding new points to a list
        plt.title("Left click for the next point or hit enter to escape")
        next_points = []
        for image, keypoints in zip(images, images_keypoints):
            plt.clf()
            plt.imshow(image, cmap = cmap)
            plt.scatter([point[0] for point in keypoints], [point[1] for point in keypoints])
            plt.draw()
            user_input = plt.ginput(1, timeout=timeout)
            if not len(user_input):
                return images_keypoints
            else:
                next_points.append(user_input[0])  # ginput returns a list of coords... want the first one
        plt.close()
        
        # If we make it through all the images, add all our keypoints to our final list
        for next_point, keypoints in zip(next_points, images_keypoints):
            keypoints.append(next_point)

def get_and_show_correspondences(images, dims, timeout = -1, include_edges = False, cmap = None, mesh = False, rainbow = True, title = None):
    """
    A wrapper for the get_correspondences that also shows images and keypoints after the fact
    Same args
    """
    images_keypoints = get_correspondences(images, dims, timeout, include_edges, cmap = cmap, mesh = mesh)
    show_correspondences(images, images_keypoints, dims, cmap, mesh, rainbow, title)
    return images_keypoints


def show_correspondences(images, images_keypoints, dims, cmap = None, 
                         mesh = False, rainbow = False, title = None, 
                         figsize = (10, 5)):
    """
    Inputs:
    images - list of images in shape (n_images, h, w, c) or (n_images, h, w)
    images_keypoints - a list of keypoints whose i th index is (n_images, n_keypoints, 2)
                        corresponding to images[i]
    dims - a tuple of dimensions for our output graphic (w, h)

    Optional arg: 
    cmap - a cmap for displaying images... useful for if the images are black and white
    mesh - Whether or not to display a mesh from the keypoints
    rainbow - whether or not to color the points with a rainbow
    title - a title for the plot
    figsize - the size of the figure to show
    """
    images_keypoints = np.array(images_keypoints)
    # Draw out the first product(*dims) images, keypoints, and triangulation
    if mesh and len(images_keypoints[0]) >= 3:
        tri = get_avg_triangulation(images_keypoints)
    fig, axs = plt.subplots(*dims, figsize = figsize)
    axs = np.array(axs)
    for ax, image, keypoints in zip(axs.flatten(), images, images_keypoints):
        ax.imshow(image, cmap = cmap)
        color = None
        if rainbow:
            color = cm.rainbow(np.linspace(0, 1, len(keypoints)))
        ax.scatter([point[0] for point in keypoints], [point[1] for point in keypoints], color = color)
        if mesh and len(images_keypoints[0]) >= 3:
            ax.triplot(keypoints[:,0], keypoints[:,1], tri.simplices)
        ax.axis("off")
    if title:
        fig.suptitle(title)
    plt.show()


# Homographies

## Imports

In [5]:
from scipy.ndimage import map_coordinates

## Code

In [6]:
def computeH(im1_pts, im2_pts):
    """
    Computes the homography from im1 to im2
    
    [         ]       [         ]
    [ im2_pts ] = H @ [ im1_pts ]
    [ 1 1 1 1 ]       [ 1 1 1 1 ]
    
    im1_pts = list of shape (pts, 2)
    im2_pts = list of shape (pts, 2)
    
    Here we use the kind of transform I talked about on my website and on the article I linked
    You can see where I define my large A matrix of augmented data, and my b vector of outputs
    """
    b = np.array([[im2_pts[i][0]] for i in range(len(im2_pts))] + 
                  [[im2_pts[i][1]] for i in range(len(im2_pts))])
    
    # Get the first half the rows of A
    # x, y, 1, 0, 0, 0, -x*x_hat, -y*x_hat
    A_1 = [[im1_pts[i][0], im1_pts[i][1], 1, 0, 0, 0,
            -im1_pts[i][0]*im2_pts[i][0], -im1_pts[i][1]*im2_pts[i][0]] 
           for i in range(len(im1_pts))]
    # Get the secong half the rows of A
    # 0, 0, 0, x, y, 1, -x*y_hat, -y*y_hat
    A_2 = [[0, 0, 0, im1_pts[i][0], im1_pts[i][1], 1,
            -im1_pts[i][0]*im2_pts[i][1], -im1_pts[i][1]*im2_pts[i][1]] 
           for i in range(len(im1_pts))]
    
    A = np.array(A_1 + A_2)
        
    H_flat = np.linalg.lstsq(A, b, rcond=None)[0]
    H_flat = H_flat.flatten()
    # Add in h_33 which is set to 1
    H_flat = np.concatenate((H_flat, [1,]))
    
    H = H_flat.reshape(3, 3)
    
    return H

"""
def computeH(im1_pts, im2_pts):
    #Computes the homography from im1 to im2
    
    #[         ]       [         ]
    #[ im2_pts ] = H @ [ im1_pts ]
    #[ 1 1 1 1 ]       [ 1 1 1 1 ]
    
    #im1_pts = list of shape (pts, 2)
    #im2_pts = list of shape (pts, 2)

    #These get im1_pts, im2_pts into the form above
    im1_pts = np.array(im1_pts).T
    im1_pts = np.concatenate((im1_pts, np.ones((1,im1_pts.shape[1]))))
    
    im2_pts = np.array(im2_pts).T
    im2_pts = np.concatenate((im2_pts, np.ones((1,im2_pts.shape[1]))))

    
    # Need to use np.solve to solve for x in b = Ax, 
    #   so need to transpose both sides, giving
    #   im2_pts.T = im1_pts.T @ H.T
    #   Now, im2_pts is b and im1_pts is a
    #   Transposing resulting H.T gives answer
    H = np.linalg.lstsq(im1_pts.T, im2_pts.T, rcond=None)[0].T
    
    # Rescale to make sure that bottom right entry is 1
    
    H = H/H[2,2]
    
    return H
"""

    
def warp_image(im, H, x_offset=0, y_offset=0, outsize = None):
    """
    Warps im by using H
    
    Values that aren't in the frame are padded with a -1 to make masks for each image easier to create
    
    x_offset, y_offset - the amount by which we shift the output up and down to put it in frame
    
    outsize - the size of the ouput (x,y) which defaults to the shape of the input image
    """
    
    if len(im.shape) == 2:
        im = np.stack([im, im, im], axis = -1)
        
        
    if outsize:
        x_coords, y_coords = np.meshgrid(range(outsize[0]), range(outsize[1]))
        out = np.zeros((outsize[1], outsize[0], 3))

    else:
        y_coords, x_coords = np.meshgrid(range(im.shape[0]), range(im.shape[1]))
        out = np.zeros(im.shape)
    
    x_coords = x_coords.flatten()
    y_coords = y_coords.flatten()
    
    coordinate_stack = np.stack([x_coords+x_offset, y_coords+y_offset, 
                                 np.ones(y_coords.shape)], axis = 0)
        
    transformed_coords = np.linalg.inv(H) @ coordinate_stack
    transformed_coords /= transformed_coords[-1,:]
        
    for channel in range(3):
        out[y_coords, x_coords, channel] = map_coordinates(im[:,:,channel], 
                                                           [transformed_coords[1], transformed_coords[0]],
                                                           mode = 'constant',
                                                           cval=-1)
    
    return out


def fast_warp_image(im, H, x_offset=0, y_offset=0):
    im = np.swapaxes(im, 0, 1)
    
    if len(im.shape) == 2:
        im = np.stack([im, im, im], axis = -1)
        
    y_coords, x_coords = np.meshgrid(range(im.shape[1]), range(im.shape[0]))
    
    map_shape = y_coords.shape
    
    x_coords = x_coords.flatten()
    y_coords = y_coords.flatten()
    
    coordinate_stack = np.stack([x_coords+x_offset, y_coords+y_offset, 
                                 np.ones(y_coords.shape)], axis = 0)
        
    transformed_coords = np.linalg.inv(H) @ coordinate_stack

    transformed_coords /= transformed_coords[-1,:]
        
    transformed_coords = transformed_coords[:-1].reshape(2, map_shape[0], map_shape[1]).astype(np.float32)

    out = cv2.remap(im, transformed_coords[1].T, transformed_coords[0].T, cv2.INTER_LINEAR, borderValue = -1)
        
    return out


def mosaic_from_left(images, keypoint_pairs = None, padding = [(0, 0), (0, 0)], middle_idx = None, correspondence_fn = get_correspondences):
    """
    Here, we take the center image in images, and gather a mosaic, returning the mosaic,
    the input images, the keypoints, and the transforms to get from one to the next
    
    Images are assumed to be from right to left
    """
    
    if not middle_idx:
        middle_idx = len(images)//2
    
    if not keypoint_pairs:
        keypoint_pairs = []

        for i in range(len(images) - 1):
            keypoint_pairs.append(correspondence_fn(images[i:i+2]))
            
    
    # This should be a list of arrays... just want to make sure we cast each pair of keypoints right
    # list of size (no_pairs, 2 (one for each image in pair), x, y)
    keypoint_pairs = [np.array(keypoint_pairs[i]) for i in range(len(keypoint_pairs))]
    
    
    images = [np.pad(image, [padding[1], padding[0], (0, 0)], 
                     mode = 'constant', constant_values = -1)
             for image in images]
    
    
    # Make sure our keypoints have their x, y values updated with low side of x, y pads
    for keypoint_pair in keypoint_pairs:
        keypoint_pair[:,:,0] += padding[0][0]
        keypoint_pair[:,:,1] += padding[1][0]
    
    
    # Do the homographies from the left of center. This is from the pair before the middle pair to the
    #    leftmost image
    #    We are going FROM the center TO the start, 
    #    so we need go go from keypoint_pair[1] -> keypoint_pair[0]
    transforms = np.identity(3)
    for i in reversed(range(middle_idx)):
        H = computeH(keypoint_pairs[i][0], keypoint_pairs[i][1])
        transforms = H @ transforms
        images[i] = fast_warp_image(images[i], transforms)
        
        
    # Do the homographies from the right of center. This is from the middle pair to the
    #    rightmost image
    #    The image we are replacing will be the i+1th image, as it is the second image in the
    #    pair
    #    We are going FROM the center TO the end, 
    #    so we need go go from keypoint_pair[0] -> keypoint_pair[1]
    transforms = np.identity(3)
    for i in range(middle_idx, len(keypoint_pairs)):
        H = computeH(keypoint_pairs[i][1], keypoint_pairs[i][0])
        transforms = H @ transforms
        images[i+1] = fast_warp_image(images[i+1], transforms)
    
    
    mask = np.sum([images[i][:,:,0] >= 0 for i in range(len(images))], axis = 0)
    mask = np.clip(mask, 1, float('inf'))
    mask = np.stack([mask, mask, mask], axis = -1)
    
    output = np.sum([np.clip(image, 0, 1) for image in images], axis = 0)
    output = output/mask
    
    return output, images, keypoint_pairs


## Results

### Basic Homography

In [31]:
mosaic1_images = [get_image(path)/256 for path in ('im1_1.jpg', 'im2_1.jpg', 'im3_1.jpg')] 
mosaic2_images = [get_image(path)/256 for path in ('im1_2.jpg', 'im2_2.jpg', 'im3_2.jpg')] 
mosaic3_images = [get_image(path)/256 for path in ('im1_3.jpg', 'im2_3.jpg', 'im3_3.jpg', 'im4_3.jpg', 'im5_3.jpg')]

In [ ]:
%matplotlib
pts_12 = get_and_show_correspondences(images[:2], (1, 2))

In [ ]:
H = computeH(_keypoint_pairs[1][1], _keypoint_pairs[1][0])
show(warp_image(images[2], H))

In [ ]:
show(images[1])

### Image Rectification

In [29]:
%matplotlib
mona_lisa_keypoints = get_and_show_correspondences([get_image('im1_1.jpg')/256], (1, 1))

Using matplotlib backend: MacOSX


In [30]:
rectify_mona_lisa_keypoints = [(0, 0), (1000, 0), (0, 1200), (1000, 1200)]

H = computeH(mona_lisa_keypoints[0], rectify_mona_lisa_keypoints)

show(warp_image(get_image('im1_1.jpg')/256, H, outsize = (1000, 1200)))

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


### Testing Merging Images (Mosaic)

In [40]:
show(mosaic_from_left(mosaic3_images[:2], keypoint_pairs = None, padding = [(0, 2000), (2000, 2000)], middle_idx = None, correspondence_fn = get_correspondences)[0])


### Bells and Whistles: Nyan Lisa

In [ ]:
%matplotlib
mona_lisa_keypoints = get_and_show_correspondences([get_image('mona_lisa.jpeg')/256], (1, 1))

In [ ]:
nyan_cat_keypoints = [(0, 0), (199, 0), (0, 250), (199, 250)]

H = computeH(nyan_cat_keypoints, mona_lisa_keypoints[0])

nyan_cat_warp = warp_image(get_image('nyan_cat.jpg')/256, H, outsize = (1000, 533))

mask = (nyan_cat_warp >=0)

In [ ]:
show(nyan_cat_warp * mask + get_image('mona_lisa.jpeg')/256 * (1 - mask))

In [ ]:
show_correspondences([get_image('mona_lisa.jpeg')/256], mona_lisa_keypoints, (1,1))

# Test

In [ ]:
show((np.ones(im1.shape) + np.logical_and(image1 >= 0, image2 >= 0))/2)

In [ ]:
show(images[1])

In [ ]:
pts1 = np.array(mona_lisa_keypoints[0]).T
pts1 = np.concatenate((pts1, np.ones((1,pts1.shape[1]))))

pts2 = np.array(rectify_mona_lisa_keypoints).T
pts2 = np.concatenate((pts2, np.ones((1,pts2.shape[1]))))



# Part B

In [ ]:
from tqdm import tqdm

In [48]:
import numpy as np
from skimage.feature import corner_harris, peak_local_max


def get_harris_corners(im, edge_discard=20):
    """
    This function takes a b&w image and an optional amount to discard
    on the edge (default is 5 pixels), and finds all harris corners
    in the image. Harris corners near the edge are discarded and the
    coordinates of the remaining corners are returned. A 2d array (h)
    containing the h value of every pixel is also returned.

    h is the same shape as the original image, im.
    coords is 2 x n (ys, xs).
    """

    assert edge_discard >= 20

    # find harris corners
    h = corner_harris(im, method='eps', sigma=1)
    coords = peak_local_max(h, min_distance=1, indices=True)

    # discard points on edge
    edge = edge_discard  # pixels
    mask = (coords[:, 0] > edge) & \
           (coords[:, 0] < im.shape[0] - edge) & \
           (coords[:, 1] > edge) & \
           (coords[:, 1] < im.shape[1] - edge)
    coords = coords[mask].T
    return h, coords


def dist2(x, c):
    """
    dist2  Calculates squared distance between two sets of points.

    Description
    D = DIST2(X, C) takes two matrices of vectors and calculates the
    squared Euclidean distance between them.  Both matrices must be of
    the same column dimension.  If X has M rows and N columns, and C has
    L rows and N columns, then the result has M rows and L columns.  The
    I, Jth entry is the  squared distance from the Ith row of X to the
    Jth row of C.

    Adapted from code by Christopher M Bishop and Ian T Nabney.
    """
    
    ndata, dimx = x.shape
    ncenters, dimc = c.shape
    assert(dimx == dimc, 'Data dimension does not match dimension of centers')

    return (np.ones((ncenters, 1)) * np.sum((x**2).T, axis=0)).T + \
            np.ones((   ndata, 1)) * np.sum((c**2).T, axis=0)    - \
            2 * np.inner(x, c)


<>:50: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:50: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/var/folders/kb/gf39c1_j7z573bq8ks8yjj_m0000gn/T/ipykernel_84993/1691399795.py:50: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(dimx == dimc, 'Data dimension does not match dimension of centers')


In [137]:
def ANMS(image, n_ip = 500, harris_threshold_scaled = .1, c_robust = .9, h = None, edge_fraction = .02):
    """
    This should return a list of coordinates for a given image
    
    Return: list of x,y coordinates, list of radius for each coordinate
    """
    # Make sure the image has 3 color channels... even if grayscale
    if len(image.shape) == 2:
        image = np.stack([image, image, image])
    
    if h is None:
        h, coords = get_harris_corners(np.mean(image, axis = 2))
        mask = np.zeros(h.shape)
        #Coords is (y, x)
        assert mask.shape == h.shape
        mask[coords[0],coords[1]] = 1
        h *= mask
    
    
    # discard points on edge
    h[-int(h.shape[0]*edge_fraction):,:] = 0
    h[:int(h.shape[0]*edge_fraction),:] = 0
    h[:,-int(h.shape[1]*edge_fraction):] = 0
    h[:,:int(h.shape[1]*edge_fraction)] = 0
    
    # Apply the thresholding to the harris detector to decrease number of points
    h = h * (h > (np.max(h) * harris_threshold_scaled))
    
    # Gets the dimensions of coordinates along 1st and second dimensions
    coords1, coords2 = np.where(h)
    
    r_vals = []
    
    for c1, c2 in tqdm(zip(coords1, coords2), total = len(coords1)):
        r = 0
        reference_h = h[c1,c2]
        while True:
            r += 1
            lower1 = max(c1 - r, 0)
            upper1 = min(c1 + r + 1, image.shape[0] - 1)
            lower2 = max(c2 - r, 0)
            upper2 = min(c2 + r + 1, image.shape[1] - 1)
            
            if True in (reference_h < c_robust * h[lower1, lower2:upper2]) \
                    or True in (reference_h < c_robust * h[upper1, lower2:upper2]) \
                    or True in (reference_h < c_robust * h[lower1:upper1, lower2]) \
                    or True in (reference_h < c_robust * h[lower1:upper1, upper2]):
                break
            
            if r > max(image.shape):
                break
            
        r_vals.append([(c1,c2),r])
    
    r_vals.sort(key = lambda x: x[-1])
    
    return np.array([(r_vals[-i][0][1], r_vals[-i][0][0]) for i in range(1, min(n_ip, len(r_vals)) + 1)]), \
            np.array([r_vals[-i][1] for i in range(1, min(n_ip, len(r_vals)) + 1)])


def feature_descriptor(image, coords, stride = 5, w_size = 8):
    if len(image.shape) == 3:
        image = np.mean(image, axis = 2)
    
    # Since we are sampling in the middle of each of the w_size x w_size patches of size stride... 
    #.    that is w_size - 1 jumps of size stride... and we want half width
    sampling_half_width = (w_size-1) * stride / 2
    
    xs = np.concatenate([np.meshgrid([coord[0] - sampling_half_width + stride * i for i in range(w_size)],
                                     [coord[1] - sampling_half_width + stride * i for i in range(w_size)])[0]
                         for coord in coords], axis = -1)
    ys = np.concatenate([np.meshgrid([coord[0] - sampling_half_width + stride * i for i in range(w_size)],
                                     [coord[1] - sampling_half_width + stride * i for i in range(w_size)])[1]
                         for coord in coords], axis = -1)
    
    #assert xs.shape[0] == xs.shape[1] and xs.shape[0] == w_size
    
    out = cv2.remap(image.astype(np.float32), xs.astype(np.float32), ys.astype(np.float32), cv2.INTER_LINEAR)
    out = np.split(out, len(coords), axis = -1)
    out = out - np.expand_dims(np.min(out, axis = (-1, -2)), [-1, -2])
    out = out / np.expand_dims(np.max(out, axis = (-1, -2)), [-1, -2])
    
    return out


def feature_matcher(features1, features2):
    """
    Returns the scores and coords respectivley for matches in features2 to each features1[i]
    """
    ssd = dist2(features1, features2)
    ssd_partition = np.partition(ssd, [0, 1], axis = 1)
    arg_partition = np.argpartition(ssd, [0, 1], axis = 1)
    return ssd_partition[:,0]/ssd_partition[:,1], arg_partition[:,:2], ssd_partition[:,0]
    
    
def ransac(image1, image2, poi_coords = None, iterations = 1000, feature_matching_threshold = .25, 
           inlier_threshold = 2, w_size = 8, stride = 5, edge_fraction = .02, n_ip = 500, min_features = 8, debug = False):

    if not poi_coords:
        # Generate our interest points. Shape (n_ip, 2)
        poi_coords1, _ = ANMS(image1, n_ip = n_ip, edge_fraction = edge_fraction)
        poi_coords2, _ = ANMS(image2, n_ip = n_ip, edge_fraction = edge_fraction)
    else:
        poi_coords1 = np.array(poi_coords[0])
        poi_coords2 = np.array(poi_coords[1])

    
    # Generate the features themselves
    features1 = feature_descriptor(image1, poi_coords1, w_size = w_size, stride = stride)
    features2 = feature_descriptor(image2, poi_coords2, w_size = w_size, stride = stride)

    # Match the features. Matches[i] should be j for features2[j] matched to features1[i]
    if debug:
        print(features1.shape, features2.shape)
    scores, two_best_matches, best_score = feature_matcher(features1.reshape(poi_coords1.shape[0], -1), features2.reshape(poi_coords1.shape[0], -1))
    matches = np.array([[i, two_best_matches[i][0]] for i in range(len(scores))])
    if debug:
        print(scores.shape, matches.shape)

    matches = matches[np.where(scores < feature_matching_threshold)]
    matched_scores = scores[np.where(scores < feature_matching_threshold)]
    
    
    # This is the case where there aren't enough features to match... return identity H and empty keypoints, inliers, and matches
    if len(set(match[1] for match in matches)) < 4:
        return np.identity(3), np.zeros((2,0,2)), np.zeros((0,2)), np.zeros((0,2))
    
    if debug:
        print(matches)
        print(poi_coords2[matches[:,1]])
 
    
    
    # Iterate a set number of times
    inliers = []
    inliers_max = 0
    while inliers_max < min_features:
        if inliers_max > 0:
            print("Running another iteration. Our maximum set of inliers is smaller than the argument min_features")
        for iteration in tqdm(range(iterations)):
            # Choose 4 matches. Pairings[0 1 2 3] is [i,j] for matched features1[i] to features2[j]
            # We also need to make sure that no two points map to the same point, since this would be a bad transformation
            pick_again = True
            while pick_again:
                pair_choices = np.random.choice(len(matches), 4)
                pairings = matches[pair_choices]
                if len(set([pairings[i,1] for i in range(4)])) == 4:
                    pick_again = False

            if iteration == 0 and debug:
                print(pair_choices.shape)
                print(pairings.shape)

            # Gets the starting coordinates for features1 (starting) and features2 (ending)
            starting_coord_idx = pairings[:,0]
            starting_coords = poi_coords1[starting_coord_idx]
            ending_coord_idx = pairings[:,1]
            ending_coords = poi_coords2[ending_coord_idx]
            if iteration == 0 and debug:
                print(starting_coord_idx.shape, starting_coord_idx)
                print(starting_coords.shape, starting_coords)
                print(ending_coord_idx.shape, ending_coord_idx)
                print(ending_coords.shape, ending_coords)

            H = computeH(starting_coords, ending_coords)
            #if iteration == 0:
            #    show(warp_image(images[0], H))

            # Order all the original point of interest coords by their idx in matches[:,0]
            all_starting_coords = np.stack([poi_coords1[:,0][matches[:,0]], poi_coords1[:,1][matches[:,0]], 
                                            np.ones(poi_coords1[:,0][matches[:,0]].shape)], axis = 0)
            # Order all the original point of interest coords by their idx in matches[:,1]
            #                                       xs                                 ys                  1s
            all_ending_coords = np.stack([poi_coords2[:,0][matches[:,1]], poi_coords2[:,1][matches[:,1]], 
                                          np.ones(poi_coords2[:,0][matches[:,1]].shape)], axis = 0)

            if iteration == 0 and debug:
                print(all_starting_coords.shape, all_ending_coords.shape)
                print(all_starting_coords[:,:5], all_ending_coords[:,:5])
            transformed_starting = H @ all_starting_coords
            transformed_starting /= transformed_starting[-1]

            distances = np.linalg.norm(all_ending_coords - transformed_starting, axis = 0)
            if iteration == 0 and debug:
                print(distances.shape, distances)

            assert distances.shape[0] == matches.shape[0]


            # We need to make sure that inliers_temp doesn't just map everything to a single coordinate... 
            #.   make sure that the size of our inliers is the number of distinct output mappings
            inliers_temp = matches[np.where(distances < inlier_threshold)]
            unique_mappings = set([inlier[1] for inlier in inliers_temp])
            if len(unique_mappings) > inliers_max:
                inliers = matches[np.where(distances < inlier_threshold)]
                inliers_max = len(unique_mappings)
    
    if debug:
        print(inliers.shape)
    starting_coord_idx = inliers[:,0]
    starting_coords = poi_coords1[starting_coord_idx]
    ending_coord_idx = inliers[:,1]
    ending_coords = poi_coords2[ending_coord_idx]
    H_final = computeH(starting_coords, ending_coords)
    
    final_keypoints = np.array([starting_coords, ending_coords])
        
    return H_final, final_keypoints, inliers, matches #, starting_coords, ending_coords, all_starting_coords, all_ending_coords


def auto_mosaic_ordered(images, padding = [(0, 0), (0, 0)], middle_idx = None, interest_points = None,
               feature_matching_threshold = .8, iterations = 50000, inlier_threshold=1, w_size = 8, stride = 5):

    if not interest_points:
        print("Getting interest points")
        interest_points = []
        for image in images:
            interest_points.append(ANMS(image, n_ip = 2000, harris_threshold_scaled = .2, edge_fraction = .02)[0])

    print("RANSACing")
    keypoint_pairs = []
    for i in range(len(images) - 1):
        _, final_keypoints, _, _ = ransac(images[i], images[i+1], poi_coords = (interest_points[i], interest_points[i+1]), feature_matching_threshold = feature_matching_threshold, 
                                          iterations = iterations, inlier_threshold = inlier_threshold, w_size = w_size, stride = stride)
        keypoint_pairs.append(final_keypoints)

    print("Stitching")
    output, images, keypoint_pairs = mosaic_from_left(images, keypoint_pairs = keypoint_pairs, padding = padding)
    
    return output, images, keypoint_pairs
    
    

Harris Edges Demo (NOTE: Images loaded from previous testing section)

In [75]:
h, coords = get_harris_corners(np.mean(mosaic1_images[0], axis = -1), edge_discard=20)
h *= h > (np.max(h) * .1)
show(h)

/var/folders/kb/gf39c1_j7z573bq8ks8yjj_m0000gn/T/ipykernel_84993/1691399795.py:21: FutureWarning: indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.
  coords = peak_local_max(h, min_distance=1, indices=True)


ANMS Demo

In [53]:
interest_points0, rs0 = ANMS(mosaic1_images[0], n_ip = 500, harris_threshold_scaled = .2, edge_fraction = .02)
interest_points1, rs1 = ANMS(mosaic1_images[1], n_ip = 500, harris_threshold_scaled = .2, edge_fraction = .02)

show_correspondences([mosaic1_images[0]], [interest_points0], dims = (1, 1))
show_correspondences([mosaic1_images[1]], [interest_points1], dims = (1, 1))

/var/folders/kb/gf39c1_j7z573bq8ks8yjj_m0000gn/T/ipykernel_84993/1691399795.py:21: FutureWarning: indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.
  coords = peak_local_max(h, min_distance=1, indices=True)
100%|██████████████████████████████████████| 4812/4812 [00:05<00:00, 809.35it/s]


Feature Descriptor Demo

In [85]:
_ = 150
show(mosaic1_images[0])
print(interest_points0[_])
show(feature_descriptor(mosaic1_images[0], np.array([interest_points0[_]])).squeeze(), cmap = 'gray')
show(mosaic1_images[0][interest_points0[_][1] - 35//2:interest_points0[_][1] + 35//2, interest_points0[_][0] - 35//2:interest_points0[_][0] + 35//2])


[2228 2256]


Feature Matching Demo

In [92]:
features1 = feature_descriptor(mosaic1_images[0], interest_points0).reshape(500, -1)
features2 = feature_descriptor(mosaic1_images[1], interest_points1).reshape(500, -1)

lowe_scores, best_second_best_args, top_scores = feature_matcher(features1, features2)

# These should be matches
_ = 150
show(mosaic1_images[0])
print(interest_points0[_])
show(features1[_].reshape(8,8), cmap = 'gray')
show(features2[best_second_best_args[_][0]].reshape(8,8), cmap = 'gray')

# These shouldn't be matches
#_ = 150
#show(features1[_].reshape(8,8), cmap = 'gray')
#show(features2[best_second_best_args[10][0]].reshape(8,8), cmap = 'gray')

# This is the lowe score for the top match
print(lowe_scores[_])


[2228 2256]
0.80562406395117 4.8598175048828125


In [107]:
# This should demo a matched feature with a low lowe score that is actually overlapped between the two images
_ = np.argmin(lowe_scores[200:]) + 200
print(_, lowe_scores[_])

show(mosaic1_images[0])
print(interest_points0[_])

show(features1[_].reshape(8,8), cmap = 'gray')
show(features2[best_second_best_args[_][0]].reshape(8,8), cmap = 'gray')
show(mosaic1_images[0][interest_points0[_][1] - 35//2:interest_points0[_][1] + 35//2, interest_points0[_][0] - 35//2:interest_points0[_][0] + 35//2])


298 0.28945643609385613
[1135 2047]


RANSAC

In [109]:
ransac

<function __main__.ransac(image1, image2, poi_coords=None, iterations=1000, feature_matching_threshold=0.25, inlier_threshold=2, w_size=8, stride=5, edge_fraction=0.02, n_ip=500, min_features=8, debug=False)>

In [111]:
#h0, _ = ANMS(images[0], n_ip = 500, harris_threshold_scaled = .1)
#h1, _ = ANMS(images[1], n_ip = 500, harris_threshold_scaled = .1)
H, keypoints12, inliers, matches = ransac(mosaic1_images[0], mosaic1_images[1], n_ip = 2000,
                                                    feature_matching_threshold = .8, iterations = 10000, inlier_threshold=1,
                                                    w_size = 8, stride = 5)
print(str(len(inliers)) + ' out of ' + str(len(matches)))

/var/folders/kb/gf39c1_j7z573bq8ks8yjj_m0000gn/T/ipykernel_84993/1691399795.py:21: FutureWarning: indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.
  coords = peak_local_max(h, min_distance=1, indices=True)
100%|███████████████████████████████████| 10000/10000 [00:03<00:00, 3276.64it/s]

16 out of 233


In [112]:
show(fast_warp_image(mosaic1_images[0], H))

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Auto Mosaic

In [124]:
#interest_points1 = []
#for image in mosaic1_images:
#    interest_points1.append(ANMS(image, n_ip = 2000, harris_threshold_scaled = .1, edge_fraction = .02)[0])
_output1, _images1, _keypoint_pairs1 = auto_mosaic_ordered(mosaic1_images, padding = [(3000, 3000), (1000, 1000)], interest_points = interest_points1)


RANSACing


100%|███████████████████████████████████| 50000/50000 [00:25<00:00, 1942.56it/s]


Stitching


In [128]:
#interest_points2 = []
#for image in mosaic2_images:
#    interest_points2.append(ANMS(image, n_ip = 2000, harris_threshold_scaled = .1, edge_fraction = .02)[0])
_output2, _images2, _keypoint_pairs2 = auto_mosaic_ordered(mosaic2_images, padding = [(3000, 3000), (1000, 1000)], interest_points = interest_points2, iterations = 200000)


RANSACing


100%|█████████████████████████████████| 200000/200000 [01:22<00:00, 2426.44it/s]


Stitching


In [149]:
mosaic3_images = [get_image(path)/256 for path in ('im1_3.jpg', 'im2_3.jpg', 'im3_3.jpg', 'im4_3.jpg', 'im5_3.jpg')]
mosaic3_images[0] = cv2.resize(mosaic3_images[0], (mosaic3_images[0].shape[1] //2, mosaic3_images[0].shape[0]//2), interpolation = cv2.INTER_AREA)
mosaic3_images[1] = cv2.resize(mosaic3_images[1], (mosaic3_images[1].shape[1] //2, mosaic3_images[1].shape[0]//2), interpolation = cv2.INTER_AREA)

interest_points3 = [[],[]]
interest_points3[0] = ANMS(mosaic3_images[0], n_ip = 500, harris_threshold_scaled = 0.05, edge_fraction = .02)[0]
interest_points3[1] = ANMS(mosaic3_images[1], n_ip = 500, harris_threshold_scaled = 0.009, edge_fraction = .02)[0]
#interest_points3[2] = ANMS(mosaic3_images[2], n_ip = 2000, harris_threshold_scaled = 0.009, edge_fraction = .02)[0]


_output3, _images3, _keypoint_pairs3 = auto_mosaic_ordered(mosaic3_images[:2], padding = [(3000, 3000), (1000, 1000)], interest_points = interest_points3[:2], iterations = 200000)


/var/folders/kb/gf39c1_j7z573bq8ks8yjj_m0000gn/T/ipykernel_84993/1691399795.py:21: FutureWarning: indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.
  coords = peak_local_max(h, min_distance=1, indices=True)
100%|█████████████████████████████████████| 1924/1924 [00:01<00:00, 1521.18it/s]


RANSACing


100%|█████████████████████████████████| 200000/200000 [01:15<00:00, 2640.48it/s]


Stitching


In [152]:
show_correspondences([mosaic3_images[0]], [interest_points3[0]], dims = (1, 1))


In [125]:
show(_output1)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [129]:
show(_output2)

In [150]:
show(_output3)

# Code Graveyard

In [ ]:
def computeH(im1_pts, im2_pts):
    """
    Computes the homography from im1 to im2
    
    [         ]       [         ]
    [ im2_pts ] = H @ [ im1_pts ]
    [ 1 1 1 1 ]       [ 1 1 1 1 ]
    
    im1_pts = list of shape (pts, 2)
    im2_pts = list of shape (pts, 2)
    """
    b = np.array([[im2_pts[i][0]] for i in range(len(im2_pts))] + 
                  [[im2_pts[i][1]] for i in range(len(im2_pts))])
    
    # Get the first half the rows of A
    # x, y, 1, 0, 0, 0, -x*x_hat, -y*x_hat
    A_1 = [[im1_pts[i][0], im1_pts[i][1], 1, 0, 0, 0,
            -im1_pts[i][0]*im2_pts[i][0], -im1_pts[i][1]*im2_pts[i][0]] 
           for i in range(len(im1_pts))]
    # Get the secong half the rows of A
    # 0, 0, 0, x, y, 1, -x*y_hat, -y*y_hat
    A_2 = [[0, 0, 0, im1_pts[i][0], im1_pts[i][1], 1,
            -im1_pts[i][0]*im2_pts[i][1], -im1_pts[i][1]*im2_pts[i][1]] 
           for i in range(len(im1_pts))]
    
    A = np.array(A_1 + A_2)
        
    H_flat = np.linalg.lstsq(A, b)[0]
    print(np.linalg.lstsq(A, b))
    print(H_flat)
    H_flat = H_flat.flatten()
    # Add in h_33 which is set to 1
    H_flat = np.concatenate((H_flat, [1,]))
    
    H = H_flat.reshape(3, 3)
    
    return H

In [ ]:
xs, ys  = np.meshgrid(range(images[0].shape[0]), range(images[0].shape[1]))
ys = ys.astype(np.float32) + 100
xs = xs.astype(np.float32) + 100

show(cv2.remap(images[0].astype(np.float32), xs, ys, cv2.INTER_LINEAR))

ValueError: too many values to unpack (expected 2)

In [70]:
show(features[0])

In [71]:
show(features[1])

In [ ]:
dist2(np.array([features[0].flatten()]), np.array([features[0].flatten()]))

In [ ]:
show(features[0])